In [1]:
!pip install -q pypdf python-dotenv transformers llama-index sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.6/276.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 74.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.9/761.9 kB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 81.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 88.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.6 MB/s eta 0

In [2]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 46.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.11-cp310-cp310-manylinux_2_35_x86_64.whl size=6423365 sha256=b8e6bd08e7fab97e066b143121a111a56ac2f1b95f1a99e4d7b0f39798fd6d60
  Stored in directory: /tmp/pip-ephem-wheel-cache-c8_xrsso/wheels/dc/42/77/a3ab0d02700427ea364de5797786c0272779dce795f62c3bc2
Successfully built llama-cpp-python


In [3]:
import logging
import sys
import torch
from llama_index import VectorStoreIndex, SimpleDirectoryReader,ServiceContext
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
from llama_index import LangchainEmbedding, ServiceContext
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

In [4]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

documents = SimpleDirectoryReader("/content/data/").load_data()

In [5]:
llm = LlamaCPP(
    model_url='https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf',
    model_path=None,
    temperature=0.1,
    context_window=3900,
    generate_kwargs={},
    model_kwargs={'n_gpu_layers':-1},
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True
)

total size (MB): 4368.44


4167it [00:29, 139.57it/s]                          
AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [9]:
embed_model = LangchainEmbedding(HuggingFaceEmbeddings(model_name="thenlper/gte-large"))
service_context = ServiceContext.from_defaults(
    chunk_size=256,
    llm=llm,
    embed_model=embed_model
)
index = VectorStoreIndex.from_documents(documents,
                                        service_context=service_context)
query_engine = index.as_query_engine()
response = query_engine.query("What are the main things to keep in mind while investing?")
print(response)

 When investing, it is important to consider different asset classes such as global equity indexes, currencies, government bonds, etc. The "state variables" to consider include the business cycle, international trade, monetary policy, and risk sentiment trends. It is also important to keep in mind the holding period, which typically ranges from three to six months.

In addition to considering different asset classes and state variables, it is also important to be aware of exogenous shocks that can impact commodity prices, such as political or geopolitical issues. Multifactor portfolios can also be constructed using country-bond ETFs.

Another strategy to consider is trading on economic announcements, such as Federal Open Market Committee (FOMC) announcements. This involves buying stocks on important announcement days and switching to risk-free assets during non-announcement days.

Finally, investing in infrastructure includes investing in long-term projects such as transportation, whic

In [10]:
while True:
  query = input()
  response = query_engine.query(query)
  print(response)

What are some techniques widely used in investing?


Llama.generate: prefix-match hit


 Some techniques widely used in investing include multifactor portfolios, trading on economic announcements, and investing in infrastructure. Multifactor portfolios typically involve using country-bond ETFs to construct a portfolio that spans the broad range of methods that can be applied to modern investment practice. Trading on economic announcements involves buying stocks on important announcement days such as Federal Open Market Committee (FOMC) announcements and switching to risk-free assets during non-announcement days. Investing in infrastructure involves investing in long-term projects such as transportation, roads, bridges, tunnels, railways, ports, airports, etc.
So for a beginner, what would be the correct approach to take while investing?


Llama.generate: prefix-match hit


 As a helpful assistant, I recommend that a beginner investor start by learning about the basics of investing and financial markets. This includes understanding different investment strategies, risk management, and diversification. It is also important to set clear financial goals and develop a long-term investment plan.

For a beginner, it may be helpful to start with simple investment strategies such as the two moving averages strategy described in the context information. This strategy involves investing in stocks based on their price relative to their moving average, and can be applied to multiple stocks. It is important to note that this strategy should be used as part of a larger investment plan and should not be relied upon solely for investment decisions.

It is also important to remember that investing always carries some level of risk, and it is important to carefully consider the potential risks and rewards before making any investment decisions. It may be helpful to consult

KeyboardInterrupt: ignored